### Setup

In [ ]:
## path
path = 'drive/MyDrive/Colab Notebooks/Zindi ASR/' # local path

#
path_data = path+'data/ASR_Zindi/'
path_save = path+ 'predictionsLM_24MayXModelsLM.csv'

# ensemble
model1_name = path+'models/Model1'
model2_name = path+'models/Model2'
model3_name = path+'models/Model3'

In [ ]:
## mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
!pip install datasets # to use
!pip install git+https://github.com/huggingface/transformers # to user huggingface transformer
!pip install jiwer # for wer metric

!pip install -U pip
!pip install -U dill
!pip install -U nltk==3.4

In [ ]:
## load packages
# standard python
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import os.path
import math
from operator import itemgetter

# preprocessing
import librosa as lb
import re
from datasets import load_metric
from sklearn.model_selection import train_test_split
from scipy.stats import entropy

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F

# transformers
from transformers import Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor

# language model
from nltk.util import pad_sequence
from nltk.util import ngrams, bigrams
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from nltk.lm.preprocessing import flatten
from nltk.lm import MLE, KneserNeyInterpolated

# nearest neighbor
import difflib

In [ ]:
## Seeding
random.seed(10)
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed_all(10)

### Data

In [ ]:
## Read data into memory (small)
df = pd.read_feather(path_data+'ASR_train_audio6683.ft')

# train valid split
df_train, df_valid = train_test_split(df, test_size=0.2, random_state=1234)

In [ ]:
## word corpus for nearest neighbor
from nltk.probability import FreqDist
from wordcloud import WordCloud, ImageColorGenerator

## Zindi dataset
words = filter(None, [re.sub('[,().?!~;1234567890^]', '', word.lower()) for word in list(df['transcription'].values)])
allwords = []

for wordlist in words:
  allwords += list(wordlist.lower().split())

# histogram. time & space complexity linear in data set size
mostcommon_small = FreqDist(allwords).most_common(1500) # it has around 1000 distinct words -> 1500 to be sure that all are included
xv, yv = zip(*mostcommon_small)

### Re-evaluate Model

In [ ]:
## Re-evaluate performance of model

# load XLSR model
if not 'XLSRmodel1' in globals():
  print('Load model')
  XLSRmodel1 = Wav2Vec2ForCTC.from_pretrained(model1_name).to("cuda")
  XLSRmodel2 = Wav2Vec2ForCTC.from_pretrained(model2_name).to("cuda")
  XLSRmodel3 = Wav2Vec2ForCTC.from_pretrained(model3_name).to("cuda") 

# load processor
if not 'processor' in globals():
  print('Load processor')
  tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("facebook/wav2vec2-large-xlsr-53-french")
  feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
  processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# prepare dataset
def prepare_dataset(batch):
    return processor(batch, return_tensors="pt", sampling_rate=16*1e3)
input_dict = df_valid['audio_signal'].apply(prepare_dataset)

# word error rate
wer_metric = load_metric("wer")
wer_ = []

# entropy
softmax = nn.Softmax(dim=2)
entropy_ = []

# WER over everything (one long string)
label_str = ''
pred_str = ''

for idx in range(len(df_valid)):
  #print('-----------------')
  logits = XLSRmodel3(input_dict.values[idx].input_values.to("cuda")).logits

  pred_ids = torch.argmax(logits, dim=-1)[0]

  # WER over everything (one long string)
  pred_str+= processor.decode(pred_ids)+ ' '
  label_str+= df_valid["transcription"].values[idx].lower()+ ' '

  # entropy
  entropy_.append(np.mean(entropy(softmax(logits).detach().to("cpu")[0].numpy())))

wer_.append(wer_metric.compute(predictions=[pred_str], references=[label_str]))

print(np.mean(wer_))
print(np.mean(entropy_))

Load model
Load processor
0.040309350831966254
1.6866072


### Beam Search

In [ ]:
## Beam Search
def beam_search_decoder(predictions, top_k = 3):
    #start with an empty sequence with zero score
    output_sequences = [([], 0)]
    
    #looping through all the predictions
    for token_probs in predictions:
        new_sequences = []
        
        #append new tokens to old sequences and re-score
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                #considering log-likelihood for scoring
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
                
        #sort all new sequences in the de-creasing order of their score
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        
        #select top-k based on score 
        # *Note- best sequence is with the highest score
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

In [ ]:
## Test beam search 
idx = 10
nbeams = 10
softmax = nn.Softmax(dim=2)

#
pred = []
input_dict = df_train['audio_signal'][idx:idx+1].apply(prepare_dataset)

for idx in range(len(input_dict)):
  #print('-----------------')
  logits = XLSRmodel1(input_dict.values[idx].input_values.to("cuda")).logits
  # sum_j(output_ij) = 1 where i is column and j is row
  output = softmax(logits) # logits -> probabilities

  beams_int = beam_search_decoder(torch.squeeze(output).tolist(), top_k = nbeams) # beams
  beams_str = nbeams*['']

  for k in range(nbeams):
    pred_ids, pred_prob = beams_int[k]
    print(processor.decode(pred_ids))

marché yeumbeul laa bëgg dem
marché yeumbeul laa bëgg dem
marché yeumbeul laa bëgg dem
marché yeumbeul laa bëgg dem
marché yeumbeul laa bëgg dem
marché yeumbeul laa bëgg dem
marché yeumbeul laa bëgg dem
marché yeumbeul laa bëgg dem
marché yeumbeul laa bëgg dem
marché yeumbeul laa bëgg dem


### Word Level Language Model

In [ ]:
## Fitting of n-gram LM
def train_ngram(LMmodel, data):
  '''
  input: model, list of sentences
  output: trained model
  '''
  #
  sentence_list = [sentence for sentence in data]

  # lower casing
  word_list_lower = [list(filter(None, [re.sub('[,().?!~;1234567890^]', '', word.lower()) for word in sentence.split(' ')])) for sentence in sentence_list]

  # preprocess for language model
  train_data, padded_words = padded_everygram_pipeline(3, word_list_lower)
  
  # fit model
  LMmodel.fit(train_data, padded_words)

  return LMmodel

In [ ]:
## Get perplexity of sentence
def ngram_logprobability(sentence):
    log_prob = 0
    count = 0
    for ngram in sentence:
      # there is probably an error in the implementation of KneserNeyInterpolated -> revert to MLE
      '''try:
        log_prob += np.log(LMKmodel.score(ngram[2], [ngram[0], ngram[1]]))
      except:
      '''
      # to avoid log(0) for unknown chars => many methods exist in the literature such as smoothing
      log_prob += np.log(LMmodel.score(ngram[2], [ngram[0], ngram[1]])+ 1e-8)
      count += 1
    return np.power(np.exp(log_prob), 1/count) # (inverse) perplexity to account for different word/ sentence length

In [ ]:
## Get perplexity of list of sentences
def prob_sentences(sentences):
  '''
  input: list of sentences
  output: log probability for sentences
  '''
  # list with log probabilities
  log_probs = len(sentences)* [-np.infty]

  # creating list of sentences from string
  list_sentences = [sentence.split(' ') for sentence in sentences]

  # lower casing
  for k in range(len(list_sentences)):
    list_sentences[k] = [word.lower() for word in list_sentences[k]]

  # list(sentence_list(word_list(ngrams)))
  list_ngrams = [list(ngrams(pad_both_ends(sentence, n=3), n=3)) for sentence in list_sentences]

  for k, sentence in enumerate(list_ngrams):
    log_probs[k] = ngram_logprobability(sentence)

  return log_probs

In [ ]:
## Language model
LMKmodel = KneserNeyInterpolated(3) # Lets train a n-gram model KneserNeyInterpolated(n)
LMKmodel = train_ngram(LMKmodel, df_train['transcription'].values)

# since there seems to be an error in the implementation of KneyerNey leading to errors -> fallback to standard MLE
LMmodel = MLE(3) # Lets train a n-gram model KneserNeyInterpolated(n)
LMmodel = train_ngram(LMmodel, df_train['transcription'].values)

### Validation

In [ ]:
## Predictions
nbeams = 20 # this should scale roughly linearly to runtime -> with 10 runs for about 10 mins
alpha = 0.

# pre-processing of data
input_dict = df_valid['audio_signal'].apply(prepare_dataset)

# word error rate
wer_ = []

#
softmax = nn.Softmax(dim=2)

# strings to calculate wer
label_str = ''
pred_str = ''

# loop through validation set
for idx in range(len(df_valid)):
  # Model 1 --------
  logits = XLSRmodel1(input_dict.values[idx].input_values.to("cuda")).logits
  output = softmax(logits) # logits -> probabilities

  # beam search
  beams_int = beam_search_decoder(torch.squeeze(output).tolist(), top_k = nbeams) # beams
  beams_str = 3*nbeams*['']
  beams_XLSR_prob = 3*nbeams*[0]

  for k in range(nbeams):
    pred_ids, pred_prob = beams_int[k]
    beams_XLSR_prob[k] = np.exp(pred_prob)
    beams_str[k] = processor.decode(pred_ids)

  ## Model 2 --------
  logits = XLSRmodel2(input_dict.values[idx].input_values.to("cuda")).logits
  output = softmax(logits) # logits -> probabilities

  # beam search
  beams_int = beam_search_decoder(torch.squeeze(output).tolist(), top_k = nbeams) # beams

  for k in range(nbeams):
    pred_ids, pred_prob = beams_int[k]
    beams_XLSR_prob[nbeams+ k] = np.exp(pred_prob)
    beams_str[nbeams+ k] = processor.decode(pred_ids)

  ## Model 3 --------
  logits = XLSRmodel3(input_dict.values[idx].input_values.to("cuda")).logits
  output = softmax(logits) # logits -> probabilities

  # beam search
  beams_int = beam_search_decoder(torch.squeeze(output).tolist(), top_k = nbeams) # beams

  for k in range(nbeams):
    pred_ids, pred_prob = beams_int[k]
    beams_XLSR_prob[2*nbeams+ k] = np.exp(pred_prob)
    beams_str[2*nbeams+ k] = processor.decode(pred_ids)

  if idx% 100 == 0:
    print(idx)

  # max prediction
  beams_LM_prob = prob_sentences(beams_str)
  pred_ = beams_str[np.argmax([pLM+alpha*pXLSR for pLM, pXLSR in zip(beams_LM_prob, beams_XLSR_prob)])].replace('<unk>', '')

  # nearest neighbor
  pred = []
  for word in pred_.split(' '):
    sim = difflib.get_close_matches(word, xv, n=1)

    if sim == []:
      sim = word
    else:
      sim = sim[0]

    pred.append(sim)
  
  # append to prediction
  pred_str += ' '.join(pred)+ ' '
  label_str+= df_valid["transcription"].values[idx].lower()+ ' '
  
# calculate wer
wer_.append(wer_metric.compute(predictions=[pred_str], references=[label_str]))
print(np.mean(wer_))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
0.01687368174361378


### Prediction

In [ ]:
## prediction
# load data (dataframe) -> empty entries
df_test = pd.read_feather(path_data+ 'ASR_test_audio1564.ft')
df_test = df_test[['ID', 'audio_signal']]
df_test.head()

def prepare_dataset(batch):
    return processor(batch, return_tensors="pt", sampling_rate=16*1e3)

In [17]:
# run through pre-processor
input_dict = df_test['audio_signal'].apply(prepare_dataset)

# to store predictions
preds = []

# beams
nbeams = 20
alpha = 0.

#
softmax = nn.Softmax(dim=2)

# run through model and decoder
for idx in range(len(df_test)):
  ## Model 1
  logits = XLSRmodel1(input_dict.values[idx].input_values.to("cuda")).logits
  # sum_j(output_ij) = 1 where i is column and j is row
  output = softmax(logits) # logits -> probabilities

  # beam search
  beams_int = beam_search_decoder(torch.squeeze(output).tolist(), top_k = nbeams) # beams
  beams_str = 3*nbeams*['']
  beams_XLSR_prob = 3*nbeams*[0]

  for k in range(nbeams):
    pred_ids, pred_prob = beams_int[k]
    beams_XLSR_prob[k] = np.exp(pred_prob)
    beams_str[k] = processor.decode(pred_ids)

  ## Model 2
  logits = XLSRmodel2(input_dict.values[idx].input_values.to("cuda")).logits
  # sum_j(output_ij) = 1 where i is column and j is row
  output = softmax(logits) # logits -> probabilities

  # beam search
  beams_int = beam_search_decoder(torch.squeeze(output).tolist(), top_k = nbeams) # beams

  for k in range(nbeams):
    pred_ids, pred_prob = beams_int[k]
    beams_XLSR_prob[nbeams+ k] = np.exp(pred_prob)
    beams_str[nbeams+ k] = processor.decode(pred_ids)

  ## Model 3
  logits = XLSRmodel3(input_dict.values[idx].input_values.to("cuda")).logits
  # sum_j(output_ij) = 1 where i is column and j is row
  output = softmax(logits) # logits -> probabilities

  # beam search
  beams_int = beam_search_decoder(torch.squeeze(output).tolist(), top_k = nbeams) # beams

  for k in range(nbeams):
    pred_ids, pred_prob = beams_int[k]
    beams_XLSR_prob[2*nbeams+ k] = np.exp(pred_prob)
    beams_str[2*nbeams+ k] = processor.decode(pred_ids)

  if idx% 100 == 0:
    print(idx)

  # append prediction
  #pred_str+= ' '.join(pred).replace('<unk>', '')+ ' '
  beams_LM_prob = prob_sentences(beams_str)
  pred_str = beams_str[np.argmax([pLM+alpha*pXLSR for pLM, pXLSR in zip(beams_LM_prob, beams_XLSR_prob)])].replace('<unk>', '')

  # nearest neighbor
  pred = []
  for word in pred_str.split(' '):
    sim = difflib.get_close_matches(word, xv, n=1)

    if sim == []:
      sim = word
    else:
      sim = sim[0]

    pred.append(sim)
  
  # append to prediction
  preds.append(' '.join(pred).replace('"', ''))
  #preds.append(pred_str)

# save as csv
dfpred = pd.DataFrame(list(zip(list(df_test['ID'].values), preds)), columns=['ID', 'transcription'])
dfpred.to_csv(path_save, index=False)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
